# Capstone Week 3
Install and Import pgeocode package to retrieve coordinates of Canada Postcodes

Install and import libxml2, libxslt, lxml to read html pages

In [1]:
%%capture
! pip install pgeocode

In [2]:
# read pages into 
import pandas as pd

html_string = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

!conda install libxml2 --yes
!conda install libxslt --yes
!conda install lxml --yes

import lxml

raw_html = pd.read_html(html_string)
postcodes = raw_html[0]
postcodes = postcodes.loc[:,['Postal Code','Borough','Neighborhood']]
postcodes_ass = postcodes[postcodes.Borough != 'Not assigned']
postcodes_ass.dropna()
postcodes_ass.head()

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - libxml2


The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2020.4.5~ --> pkgs/main::ca-certificates-2020.1.1-0
  certifi            conda-forge::certifi-2020.4.5.1-py36h~ --> pkgs/main::certifi-2020.4.5.1-py36_0
  openssl            conda-forge::openssl-1.1.1g-h516909a_0 --> pkgs/main::openssl-1.1.1g-h7b6447c_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [3]:
postcodes_ass.shape

(103, 3)

In [4]:
# %%capture
# ! pip install geopandas

# pgeocode packages

pgeocode is straightforward method of retrievng the coordinates of all the postcodes

After some data wrangling merge the postcodes with the retrieved coords.

In [5]:
import pgeocode

nomi = pgeocode.Nominatim('ca')

#loop until you get the coordinates
coords = nomi.query_postal_code(postcodes_ass.loc[:,'Postal Code'].tolist())
coords.dropna()
coords.head()
coords = coords[['postal_code','latitude','longitude']]
coords.rename(columns={'postal_code':'Postal Code',
                          'latitude':'Latitude',
                          'longitude':'Longitude'}, 
                 inplace=True)
pc_concat = pd.merge(postcodes_ass,coords, on= 'Postal Code')
pc_concat.dropna(inplace=True)
pc_concat.tail(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
93,M8W,Etobicoke,"Alderwood, Long Branch",43.6021,-79.5402
94,M9W,Etobicoke,"Northwest, West Humber - Clairville",43.7144,-79.5909
95,M1X,Scarborough,Upper Rouge,43.8340,-79.2069
96,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.6684,-79.3689
97,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.6492,-79.3823
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6518,-79.5076
99,M4Y,Downtown Toronto,Church and Wellesley,43.6656,-79.3830
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.7804,-79.2505
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6325,-79.4939
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.6256,-79.5231


In [6]:
%%capture
! pip install geocoder -v
! pip install geopy
! pip install geopandas --verbose

# Folium Maps

In [7]:
!conda install -c conda-forge folium=0.11.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium=0.11.0


The following packages will be UPDATED:

  ca-certificates     pkgs/main::ca-certificates-2020.1.1-0 --> conda-forge::ca-certificates-2020.4.5.1-hecc5488_0

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi              pkgs/main::certifi-2020.4.5.1-py36_0 --> conda-forge::certifi-2020.4.5.1-py36h9f0ad1d_0
  openssl              pkgs/main::openssl-1.1.1g-h7b6447c_0 --> conda-forge::openssl-1.1.1g-h516909a_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done


Import Nominatim, geopandas, geopy
Dropped google's geolocation service as it was denying requests
Used Nominatim instead

In [8]:
# Let's starting putting the dots on the map
# define the Toronto map
import geocoder
import folium
import geopandas
import geopy

from geopy.geocoders import Nominatim
locator = Nominatim(user_agent='myGeocoder')
location = locator.geocode('Toronto, Ontario')

markers = folium.map.FeatureGroup()
pc_concat.shape

(102, 5)

In [9]:
pc_concat.dropna(inplace=True)
pc_concat.isnull().sum().sum()

0

In [21]:
# loop through the postcode and add each to the markers feature group
for lab, lat, lng, in zip(pc_concat['Postal Code'],pc_concat.Latitude, pc_concat.Longitude):
    markers.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6,
            popup = lab
        )
    )
    
toron_map = folium.Map(location=[location.latitude, location.longitude], zoom_start=9)
markers.add_to(toron_map)
toron_map

'M8Z'